jupyter notebook不能直接识别CUDA，需要以下准备工作：
1. pip install ipykernel
2. python -m ipykernel install --name pytorch   # pytorch为自定义的内核名

在new打开新的.ipynb时不选择python3而是选择上述自定义的内核pytorch即可

In [1]:
!nvidia-smi

Wed Apr 19 21:22:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.31       Driver Version: 462.31       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 106... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   54C    P8     4W /  N/A |    237MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import torch

# 查看是否支持CUDA
print("是否支持GPU：", torch.cuda.is_available())
print("可用的GPU数量：", torch.cuda.device_count())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device：", device)
print("当前设备索引：", torch.cuda.current_device())

None
是否支持GPU： True
可用的GPU数量： 1
device： cuda
当前设备索引： 0


## 随机抽样函数

1. torch.seed()：生成不确定的随机数

In [3]:
torch.seed()

663069032300

2. torch.manual_seed(seed)：设定生成随机数的种子，返回一个torch.Generator对象

      参数：种子seed为int或long类型，生成的随机数是固定的

In [4]:
# 为CPU设置种子，生成随机数
print(torch.manual_seed(1))
print(torch.manual_seed(1))

In [6]:
# 为GPU设置种子，生成随机数
print(torch.cuda.manual_seed(1000))
print(torch.rand(1, 2))

tensor([[0.6514, 0.1827]])


In [41]:
# 为所有GPU设置种子
print(torch.cuda.manual_seed_all(2023))

None


In [29]:
# 不设置随机种子，生成随机数，每次运行结果都不一样
torch.manual_seed(2023)
print(torch.rand(1))

tensor([0.4290])


GPU的种子设置好像有有问题，随机数并未固定不变

3. torch.initial_seed()：返回生成随机数的原始种子值

In [42]:
print(torch.initial_seed())
print(torch.cuda.initial_seed())

2023
2023


## Tensor的创建方法

1. torch.tensor()直接创建

   torch.tensor(data, dtype=None, device=None, requires_grad=False, pin_memory=False)
   
   &data: 数据，可以是list或numpy
   
   &dtype: 数据类型，默认与data保持一致
   
   &device: 张量所在的设备，cuda或cpu
   
   &requires_grad: 是否需要梯度
   
   &pin_memory: 是否存于锁存内存，锁页内存，设置为True时将内存的Tensor转义到GPU的显存会更快一些
   

In [7]:
import numpy as np

arr = np.ones((3, 3))
print("原始数据类型：", arr.dtype)
t = torch.tensor(arr)
print(t)

原始数据类型： float64
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)


In [8]:
# 若直接一个放在GPU中的数据
tt = torch.tensor(arr, device=device)
print(tt)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], device='cuda:0', dtype=torch.float64)


2. torch.from_numpy(ndarray)：从numpy创建tensor

   该方法创建的tensor和原来的ndarray共享内存，修改tensor，ndarray中的数据页也会被修改

In [10]:
arr = np.array([[1, 2, 3], [4, 5, 6]])

t = torch.from_numpy(arr)
print("原始tensor: ", t)
t[0, 0] = -1
print("原始numpy数组：", arr)
print("修改后tensor：", t)

原始tensor:  tensor([[1, 2, 3],
        [4, 5, 6]], dtype=torch.int32)
原始numpy数组： [[-1  2  3]
 [ 4  5  6]]
修改后tensor： tensor([[-1,  2,  3],
        [ 4,  5,  6]], dtype=torch.int32)


In [17]:
# 将tensor设置到CUDA设备,to(device)不是in-place操作
print(t.to("cuda"))
print(t)

tensor([[-1,  2,  3],
        [ 4,  5,  6]], device='cuda:0', dtype=torch.int32)
tensor([[-1,  2,  3],
        [ 4,  5,  6]], dtype=torch.int32)


3. torch.zeros()：根据size创建全0张量

    torch.zeros(*size, out=None, dtype=None, layout=torch.strided, device=None, requires_grad=False)
    
    &size: 张量的形状
    
    &out: 输出的张量，如果指定了out，返回的张量和out指向同一个地址
    
    &layout: 内存中的布局形式，有strided和sparse_coo等，当是稀疏矩阵时设置为sparse_coo可以减少内存占用

In [20]:
out_t = torch.tensor([1])
print(f'out_t: {out_t}')

t = torch.zeros((3, 2), out=out_t)

print(f't: {t}', '\n', f'out_t: {out_t}')
print(id(t), id(out_t), id(t) == id(out_t))

out_t: tensor([1])
t: tensor([[0, 0],
        [0, 0],
        [0, 0]]) 
 out_t: tensor([[0, 0],
        [0, 0],
        [0, 0]])
2803705758640 2803705758640 True


4. torch.zeros_like()：根据input形状创建全0的张量

    torch.zeros_like(input, dtype=None, device=None, requires_grad=False, memory_format=torch.preserve_format)
   
   类似的还有torch.ones() torch.ones_like()
    

In [21]:
print(torch.ones_like(t))

tensor([[1, 1],
        [1, 1],
        [1, 1]])


5. torch.full(),torch.full_like(): 创建自定义某一数值的张量

    torch.full(size, fill_value, out=None, dtype=None, layout=torch.strided, device=None, requires_grad=False)
    
    &size: 张量的形状
    &fill_value: 张量中每一个元素的值

In [23]:
t = torch.full((3, 4), 6)
print(t)

tensor([[6, 6, 6, 6],
        [6, 6, 6, 6],
        [6, 6, 6, 6]])


6. torch.arange()：创建等差的一维张量

    torch.arange(start=0, end, step=1, out=None, dtype=None, layout=torch.strided, device=None, requires_grad=False)
    
    &start: 数列起始值，默认为0
    
    &end: 数列结束值，开区间，取不到结束值
    
    &step: 数列公差，默认为1
    

In [24]:
torch.arange(2, 10, 2)

tensor([2, 4, 6, 8])

7. torch.linspace()：创建均分的一维张量

    torch.linspace(start, end, steps=100, out=None, dtype=None, layout=torch.strided, device=None, requires_grad=False)
    
    &steps: 数列长度（元素个数）

In [25]:
torch.linspace(2, 10, 6)

tensor([ 2.0000,  3.6000,  5.2000,  6.8000,  8.4000, 10.0000])

8. torch.logspace()：创建对数均分的一维张量

    &steps: 数列长度
    &base: 对数的底，默认为10

In [26]:
print(torch.logspace(2, 5, 3))

tensor([   100.0000,   3162.2776, 100000.0000])


9. torch.eye()：创建单位对角矩阵

    torch.eye(n, m=None, out=None, dtype=None, layout=torch.strided, device=None, requires_grad=False)
    
    &n: 矩阵行数，通常只设置n，为方针
    &m: 矩阵列数

In [27]:
print(torch.eye(5))
print(torch.eye(5, 3))

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 0.],
        [0., 0., 0.]])


### 根据概率创建Tensor

1. torch.normal(): 生成正态分布（高斯分布），返回一个张量，包含从给定参数means,std的离散正态分布中抽取随机数。
    
    torch.normal(mean, std, *, generator=None, out=None）
    
    &mean: 均值
    &std: 标准差

In [28]:
# mean和std均为标量
print(torch.normal(0., 1., size=(3, 4)))

tensor([[ 0.5867, -0.4719, -0.6534, -0.9926],
        [-0.0737, -0.7257,  0.5127, -0.3303],
        [-0.1518, -1.6457, -0.1259,  0.1261]])


In [30]:
# mean为张量，std为标量
print(torch.normal(torch.arange(1, 5, 1, dtype=torch.float), 1))

tensor([1.1194, 0.9597, 2.8995, 3.9450])


In [31]:
# mean和std均为张量
print(torch.normal(torch.arange(1, 5, 1, dtype=torch.float), torch.arange(1, 5, 1, dtype=torch.float)))

tensor([1.1866, 0.1724, 3.7337, 1.7283])


2. torch.rand()和torch.rand_like(): 在区间上生成均匀分布

    torch.rand(*size, out=None, dtype=None, layout=torch.strided, device=None, requires_grad=False)

In [32]:
print(torch.rand(3, 4))

tensor([[0.3669, 0.6483, 0.9769, 0.1550],
        [0.7877, 0.5785, 0.3953, 0.3998],
        [0.2483, 0.1760, 0.4140, 0.3999]])


3. torch.randint()和torch.randint_like()：左开右闭区间，生成整数均匀分布

    torch.randint(low=0, high, size, *, generator=None, out=None, dtype=None, layout=torch.strided, device=None, requires_grad=False)

In [33]:
print(torch.randint(10, 100, (4, 5)))

tensor([[14, 59, 39, 41, 83],
        [63, 26, 99, 92, 44],
        [51, 17, 16, 51, 58],
        [54, 82, 35, 61, 48]])


4. torch.randperm()：生成0到n-1的随机排列

In [37]:
print(torch.randperm(4))

tensor([3, 0, 2, 1])


5. torch.bernoulli(): 以input为概率，生成伯努利分布（0-1分布，两点分布）

    torch.bernoulli(input, *, generator=None, out=None)

In [44]:
print(torch.bernoulli(torch.randint(10, 100, (4, 5))))

RuntimeError: "bernoulli_tensor_cpu_p_" not implemented for 'Long'

In [49]:
print(torch.empty(3, 3).uniform_(0, 1))

tensor([[0.3043, 0.1039, 0.6472],
        [0.0592, 0.0811, 0.6778],
        [0.9146, 0.7838, 0.1126]])


In [56]:

print(torch.zeros(3, 3))
print(torch.empty(3, 3))
torch.set_printoptions(precision=36)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
